## **Structural Mapping of Protein Mutations**


#Step 1. Install the required modules

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.5 MB/s eta 0:00:00


In [18]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.3 MB/s eta 0:00:00


In [2]:
!apt-get install -y clustalo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libargtable2-0
The following NEW packages will be installed:
  clustalo libargtable2-0
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 275 kB of archives.
After this operation, 692 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libargtable2-0 amd64 13-1 [13.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 clustalo amd64 1.2.4-4build1 [262 kB]
Fetched 275 kB in 2s (180 kB/s)
Selecting previously unselected package libargtable2-0.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../libargtable2-0_13-1_amd64.deb ...
Unpacking libargtable2-0 (13-1) ...
Selecting previously unselected package clustalo.
Preparing to unpack .../clustalo_1.2.4-4build1_amd64.deb ...
Unpacking clustalo (1.2.4-4build1) ...

In [3]:
import os
from Bio import SeqIO
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.Align.Applications import ClustalOmegaCommandline
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from google.colab import files
import shutil

**If you want to import files from your google drive, execute the following command.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Step 2. Nucleotide Translation
If you are starting your analysis from DNA sequences execute the following commands. **The reference sequence for conservation analysis must be placed as the first in the fasta file.**

You can create a new directory in Google Colab folder or use your own Google drive directory.

**If using colab folders, all the outputs will be inputs for the next steps (recommended):**

Create a folder named "nucseq" in the content directory

In [ ]:
#Replace input_dir and output_dir as required.
input_dir = "/content/nucseq"
output_dir = "/content/protseq"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def translate_sequence(nucleotide_seq):
    amino_acid_seq = nucleotide_seq.translate()
    return amino_acid_seq

for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)
        nucleotide_seqs = SeqIO.parse(input_path,"fasta")

        amino_acid_seqs = []
        for seq_record in nucleotide_seqs:
            amino_acid_seq = translate_sequence(seq_record.seq)
            amino_acid_record = seq_record
            amino_acid_record.seq = amino_acid_seq
            amino_acid_seqs.append(amino_acid_record)
            output_filename = os.path.splitext(filename)[0] + "_prot.fasta"
        output_path = os.path.join(output_dir, output_filename)
        SeqIO.write(amino_acid_seqs, output_path, "fasta")

/usr/local/lib/python3.9/dist-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


#Step 3. Protein Alignment
This step will perform the protein alignment using the output from the first step. The code will identify, list and exclude from alignment, all the entries containing stop codons in the middle of the sequence. **The reference sequence for conservation analysis must be placed as the first in the fasta file (if not inserted in the previous step).**

In [4]:
##Replace input_dir and output_dir as required.
input_dir = "/content/protseq"
output_dir = "/content/aln"

# create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# iterate over all files in the input directory
for file in os.listdir(input_dir):
    # check if file is a fasta file
    if file.endswith(".fasta"):
        input_path = os.path.join(input_dir, file)
        output_path = os.path.join(output_dir, file)

        # filter out sequences containing stop codons before the last residue
        sequences = []
        excluded_sequences = []
        for record in SeqIO.parse(input_path, "fasta"):
            sequence = str(record.seq)
            if "*" not in sequence[:-1]:  # check if the last residue is not a stop codon
                sequences.append(record)
            else:
                excluded_sequences.append(record.id)

        # run Clustal Omega for non-excluded sequences and wait for it to finish
        SeqIO.write(sequences, input_path, "fasta")  # overwrite input file with non-excluded sequences
        clustalomega_cline = ClustalOmegaCommandline(infile=input_path, outfile=output_path, auto=True)
        stdout, stderr = clustalomega_cline()

        # parse Clustal alignment and print summary
        alignment = AlignIO.read(output_path, "fasta")
        print("Alignment of {} sequences with length {} has been created. Excluded sequences: {}".format(
            len(alignment), alignment.get_alignment_length(), ", ".join(excluded_sequences)))


Alignment of 22 sequences with length 599 has been created. Excluded sequences: 
Alignment of 20 sequences with length 515 has been created. Excluded sequences: 
Alignment of 22 sequences with length 694 has been created. Excluded sequences: 
Alignment of 19 sequences with length 763 has been created. Excluded sequences: 
Alignment of 20 sequences with length 759 has been created. Excluded sequences: 
Alignment of 16 sequences with length 757 has been created. Excluded sequences: 
Alignment of 21 sequences with length 610 has been created. Excluded sequences: 
Alignment of 19 sequences with length 601 has been created. Excluded sequences: 
Alignment of 20 sequences with length 661 has been created. Excluded sequences: 


#Step 4. Generation of Attribute Files
This step will create attribute files that can be read by Chimera allowing the structural mapping of each mutation found in the alignments. The first sequence in the alignment files will be set as the reference sequence for conservation analysis.

In [5]:
# Step 1: Define the input and output folders
input_dir = "/content/aln"
output_dir = "/content/attributes"

# create the output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Step 2: Iterate over all the files in the input folder
for file_name in os.listdir(input_dir):
    if file_name.endswith(".fasta"):
        input_file_path = os.path.join(input_dir, file_name)

        # Step 3: Read the alignment file
        alignment = AlignIO.read(input_file_path, "fasta")

        # Step 4: Get the reference sequence
        ref_seq = alignment[0].seq

        # Step 5: Calculate the identity of each residue
        output_file_name = os.path.splitext(file_name)[0] + "_attributes.txt"
        output_file_path = os.path.join(output_dir, output_file_name)

        with open(output_file_path, "w") as f:
            # Write the attribute headers
            f.write("attribute: resconservation\n")
            f.write("recipient: residues\n")
            for j, ref_aa in enumerate(ref_seq):
                if ref_aa == "-":
                    continue
                conservation_count = 0
                for record in alignment:
                    seq = str(record.seq)
                    aa = seq[j]
                    if aa == "-":
                        continue
                    if record.id != alignment[0].id and aa == ref_aa:
                        conservation_count += 1
                conservation_percentage = (conservation_count / (len(alignment)-1)) * 100 if ref_aa != '-' else 0
                residue_num = j + 1

                # Write the attribute value for this residue
                line = f"\t:{residue_num}.A\t{conservation_percentage:.2f}\n"
                f.write(line)

        print("Attribute file generated: ", output_file_path)

Attribute file generated:  /content/attributes/TP0117_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP1031_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0610_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0313_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0621_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0317_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0620_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0131_21samplesMU_prot_attributes.txt
Attribute file generated:  /content/attributes/TP0011_21samplesMU_prot_attributes.txt


#Step 5. Summary Table of identified mutations
This step will create excel sheets describing the identified mutations for each alignment as also a file combining all the data.

In [19]:
# Set the input and output directories
input_dir = "/content/aln"
output_dir = "/content/table"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate through each file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".fasta"):
        # Load the multi sequence alignment file
        alignment = AlignIO.read(os.path.join(input_dir, filename), "fasta")

        # Create an empty dictionary to store the mutation information
        mutations_dict = {}

        # Iterate through each position in the alignment and compare the amino acid residue at that position in each sequence
        for i in range(alignment.get_alignment_length()):
            position = i + 1
            residues = set(alignment[:, i])
            if len(residues) == 1:
                continue  # skip if all residues at the position are the same
            wt_residue = alignment[0, i]
            for j in range(1, len(alignment)):
                if alignment[j, i] != wt_residue:
                    mutation = f"{wt_residue}{position}{alignment[j, i]}"
                    if mutation in mutations_dict:
                        mutations_dict[mutation]["count"] += 1
                        mutations_dict[mutation]["sequences"].append(alignment[j].id)
                    else:
                        mutations_dict[mutation] = {
                            "position": position,
                            "wt_residue": wt_residue,
                            "mut_residue": alignment[j, i],
                            "count": 1,
                            "sequences": [alignment[j].id],
                        }

        # Convert the mutations dictionary to a Pandas DataFrame
        df = pd.DataFrame(mutations_dict.values(), columns=["wt_residue", "position", "mut_residue", "count", "sequences"])

        # Add a percentage column to the DataFrame
        total_count = df["count"].sum()
        df["percentage"] = (df["count"] / (len(alignment) - 1)) * 100

        # Save the DataFrame to an Excel file in the output directory
        output_filename = os.path.splitext(filename)[0] + "_mutations.xlsx"

        # Create a Pandas Excel writer
        writer = pd.ExcelWriter(os.path.join(output_dir, output_filename), engine="xlsxwriter")

        # Write the DataFrame to the Excel sheet
        df.to_excel(writer, sheet_name="mutations", index=False)

        # Get the worksheet object
        worksheet = writer.sheets["mutations"]

        # Write the sequence headers to the new column
        sequences = df["sequences"].tolist()
        sequence_column = len(df.columns) + 1  # Next column after the last column of DataFrame
        worksheet.write(0, sequence_column - 1, "Sequences")  # Write column header
        for i, seq_list in enumerate(sequences):
            worksheet.write_column(1, sequence_column - 1, seq_list)

        # Close the Pandas Excel writer
        writer.save()


<ipython-input-19-1502c4fbf61e>:67: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-19-1502c4fbf61e>:67: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-19-1502c4fbf61e>:67: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-19-1502c4fbf61e>:67: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-19-1502c4fbf61e>:67: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-19-1502c4fbf61e>:67: FutureWarning: save is not part of the public API, usage can give unexpected re

In [7]:
# Set the input directory for the mutation Excel files
input_dir = "/content/table"

# Create a new Excel file and add the first sheet
output_filename = "combined_mutations.xlsx"
wb = Workbook()
ws = wb.active

# Iterate through each file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith("_mutations.xlsx"):
        # Load the Excel file into a Pandas DataFrame
        df = pd.read_excel(os.path.join(input_dir, filename))

        # Add a new sheet to the output file with the mutations from this input file
        ws = wb.create_sheet(title=os.path.splitext(filename)[0])

        # Write the mutations DataFrame to the new sheet
        for r in dataframe_to_rows(df, index=False, header=True):
            ws.append(r)

# Save the output file
wb.save(os.path.join(input_dir, output_filename))

/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.w

#Step 6. Folders Download
This step will download all folders from google colab. You can run only the commands for the desired folders.

In [8]:
#Proteins Folder
folder_protseq = "/content/protseq"
shutil.make_archive(folder_protseq, 'zip', folder_protseq)
files.download(f"{folder_protseq}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#Alignment Folder
folder_aln = "/content/aln"
shutil.make_archive(folder_aln, 'zip', folder_aln)
files.download(f"{folder_aln}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#Attributes Folder
folder_attributes = "/content/attributes"
shutil.make_archive(folder_attributes, 'zip', folder_attributes)
files.download(f"{folder_attributes}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
#Excel Sheet Folder
folder_xlsx = "/content/table"
shutil.make_archive(folder_xlsx, 'zip', folder_xlsx)
files.download(f"{folder_xlsx}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>